# Importacion de librerias

In [87]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os
from scipy.optimize import minimize
import ipywidgets as widgets

%matplotlib inline

In [88]:
# !pip install kaleido==0.1.0

# Clases y metodos auxiliares

In [89]:
class AdjustTrajectory():
    # Nombre del archivo de parametros de los modelos no lineales para cada punto 3D
    nonlinear_models_parameters_file_name="nonlinear_models_parameters.npy"

    # Funcion de perdida a optimizar
    @staticmethod
    def L(params, point_3D_over_time):
        # Numero de puntos 3D a ajustar
        m=point_3D_over_time.shape[0]
        # Valores de t para cada punto en la parametrizacion de la curva (p(t)) (cada punto 3D tiene asociado un valor de t)
        ts=params[0:m]
        # Parametros de los polinomios de grado 2 en cada dimension 
        wx1,wx2,wy1,wy2,wz1,wz2=params[m:]
        # Punto inicial de la parametrizacion
        p0=point_3D_over_time[0]
        cost=0
        for i in range(m):
            # Un punto 3D de la nube de puntos
            u=point_3D_over_time[i]
            # Un valor de t
            t=ts[i]
            # Un punto en la parametrizacion
            p=p0 + np.array([wx1 * t + wx2 * t ** 2, wy1 * t + wy2 * t ** 2, wz1 * t + wz2 * t ** 2])
            # Se mide la distancia entre el punto 3D en la nube de puntos y el punto 3D en la parametrizacion
            cost+=np.linalg.norm(u - p) ** 2
        cost*=(1/(2 * m))
        return cost

    # Para ajustar una parametrizacion de una curva a una nube de puntos 
    @staticmethod
    def adjust(point_3D_over_time):
        # Numero de puntos 3D
        m=point_3D_over_time.shape[0]
        # Valores iniciales
        ts_initial=np.random.random((1,m))
        ws_initial=np.random.random((1,6))
        initial_params=np.concatenate([ts_initial, ws_initial], axis=1).flatten()
        # Minimizacion de la funcion de perdida
        res = minimize(lambda params: AdjustTrajectory.L(params, point_3D_over_time), initial_params, method='BFGS', jac=False)
        # Valores optimos
        ts=res.x[0:m]
        ws=res.x[m:]
        # Solo nos interesa el t minimo y el t maximo de todos los ts 
        params=np.concatenate([np.array([ts.min(), ts.max()]), ws])
        return params

    # Para predecir puntos sobre la curva ajustada
    @staticmethod
    def predict(params, p0, n):
        # Parametros optimos
        t_min,t_max=params[0:2]
        wx1,wx2,wy1,wy2,wz1,wz2=params[2:]
        # Parametrizacion de la curva ajustada
        p=lambda t: p0 + np.array([wx1 * t + wx2 * t ** 2, wy1 * t + wy2 * t ** 2, wz1 * t + wz2 * t ** 2])
        # Puntos 3D sobre la curva ajustada
        point_3D_over_time=np.concatenate([p(t)[None,:] for t in np.linspace(t_min, t_max, n)], axis=0)
        return point_3D_over_time

    # Para obtener las posturas 3D a lo largo del tiempo
    @staticmethod
    def get_posture_3D_over_time_list(specific_trajectory_full_path):
        files=os.listdir(specific_trajectory_full_path)
        if AdjustTrajectory.nonlinear_models_parameters_file_name in files:
            # Si tiene parametros
            files.remove(AdjustTrajectory.nonlinear_models_parameters_file_name)
        else:
            # No tiene parametros
            pass
        files.sort()
        # Lista de posturas 3D a lo largo del tiempo
        posture_3D_over_time_list=[np.load(os.path.join(specific_trajectory_full_path, *[files[i]])) for i in range(len(files))]
        return posture_3D_over_time_list

    # Para cargar los parametros de los modelos no lineales
    @staticmethod
    def load_nonlinear_models_parameters(specific_trajectory_full_path):
        file_path=os.path.join(specific_trajectory_full_path, *[AdjustTrajectory.nonlinear_models_parameters_file_name])
        if os.path.exists(file_path):
            nonlinear_models_parameters=np.load(file_path)
            return nonlinear_models_parameters
        else:
            return None

    # Para guardar los parametros de los modelos no lineales
    @staticmethod
    def save_nonlinear_models_parameters(specific_trajectory_full_path, nonlinear_models_parameters):
        # if not os.path.exists(specific_trajectory_full_path):
        #     os.makedirs(specific_trajectory_full_path)
        np.save(os.path.join(specific_trajectory_full_path, *[AdjustTrajectory.nonlinear_models_parameters_file_name]), nonlinear_models_parameters)

    # Para obtener las trayectorias de los puntos 3D a lo largo del tiempo
    @staticmethod
    def get_point_3D_over_time_list(posture_3D_over_time_list, algorithm_number_points):
        point_3D_over_time_list=[]
        # Para cada punto 3D
        for j in range(algorithm_number_points):
            # Trayectoria de un solo punto 
            point_3D_over_time=np.zeros((len(posture_3D_over_time_list), 3))
            for i in range(len(posture_3D_over_time_list)):
                # Punto 3D en un instante de tiempo para un punto 3D especifico
                point_3D_over_time[i]=posture_3D_over_time_list[i][j]
            point_3D_over_time_list.append(point_3D_over_time)
        return point_3D_over_time_list

    # Para calcular los parametros de los modelos no lineales para cada punto 3D
    @staticmethod
    def calculate_nonlinear_models_parameters(posture_3D_over_time_list, algorithm_number_points):
        nonlinear_models_parameters=np.zeros((algorithm_number_points, 8))
        point_3D_over_time_list=AdjustTrajectory.get_point_3D_over_time_list(posture_3D_over_time_list, algorithm_number_points)
        for i in range(algorithm_number_points):
            nonlinear_models_parameters[i]=AdjustTrajectory.adjust(point_3D_over_time=point_3D_over_time_list[i])
        return nonlinear_models_parameters

    # Para ajustar posturas 3D y trayectorias de los puntos 3D a lo largo del tiempo
    @staticmethod
    def get_adjusted_posture_3D_and_point_3D_over_time_list(posture_3D_over_time_list, nonlinear_models_parameters, algorithm_number_points, n=100):
        # n: Numero de puntos que se obtendran a lo largo de la trayectoria ajustada
        adjusted_posture_3D_over_time_list=[np.zeros((algorithm_number_points,3)) for i in range(n)]
        adjusted_point_3D_over_time_list=[]
        # Para cada punto 3D
        for i in range(algorithm_number_points):
            # Trayectoria ajustada de un solo punto 
            adjusted_point_3D_over_time=AdjustTrajectory.predict(params=nonlinear_models_parameters[i], p0=posture_3D_over_time_list[0][i], n=n)
            adjusted_point_3D_over_time_list.append(adjusted_point_3D_over_time)
            for j in range(n):
                # Punto 3D en un instante de tiempo para un punto 3D especifico
                adjusted_posture_3D_over_time_list[j][i]=adjusted_point_3D_over_time[j]
        return adjusted_posture_3D_over_time_list,adjusted_point_3D_over_time_list

In [90]:
# Para graficar posturas 3D y la trayectoria ajustada de los puntos 3D a lo largo del tiempo (opcional)
def get_posture_figure(posture_3D_over_time, algorithm_point_names_list, algorithm_connection_list, adjusted_point_3D_over_time_list=None, rotation=1.25, title=""):
    # Puntos 3D
    data=[
        go.Scatter3d(
            x=posture_3D_over_time[:,0], 
            y=posture_3D_over_time[:,1], 
            z=posture_3D_over_time[:,2], 
            mode="markers", 
            name="",
            marker=dict(size=6, symbol="circle", color="red"),
            text=[f"{algorithm_point_names_list[i]} ({i})" for i in range(len(algorithm_point_names_list))]
        )
    ]
    # Conexion entre puntos 3D
    for connection in algorithm_connection_list:
        c1,c2=connection
        x1,y1,z1=posture_3D_over_time[c1]
        x2,y2,z2=posture_3D_over_time[c2]
        data.append(
            go.Scatter3d(
                x=[x1,x2], 
                y=[y1,y2], 
                z=[z1,z2], 
                mode="lines", 
                name="",
                line=dict(width=5, color='green')
            )
        )
    if adjusted_point_3D_over_time_list is not None:
        # Trayectoria ajustada de los puntos 3D a lo largo del tiempo
        for adjusted_point_3D_over_time in adjusted_point_3D_over_time_list:
            data.append(
                go.Scatter3d(
                    x=adjusted_point_3D_over_time[:,0], 
                    y=adjusted_point_3D_over_time[:,1], 
                    z=adjusted_point_3D_over_time[:,2], 
                    mode='lines', 
                    name='Trajectory',
                    line=dict(width=3, color='blue')
                )
            )
    fig = go.Figure(data=data)
    fig.update_layout(
        scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title='z',
            xaxis = dict(nticks=4, range=[-1.5,1.5]),
            yaxis = dict(nticks=4, range=[-1.5,1.5]),
            zaxis = dict(nticks=4, range=[-1.5,1.5]),
            aspectratio=dict(x=1, y=1, z=1)
        ),
        title=dict(text=title), 
        autosize=False,
        width=800, 
        height=500,
        margin=dict(l=40, r=40, b=40, t=40),
        showlegend=False,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            font=dict(
                family="Courier",
                size=14,
                color="black"
            ),
            itemsizing="constant"
        ),
        scene_camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=rotation, y=1.25, z=1.25)
        )
    )
    return fig

In [91]:
# Para graficar la trayectoria ajustada de un punto 3D a lo largo del tiempo 
def get_trajectory_figure(point_3D_over_time, adjusted_point_3D_over_time, rotation=1.25, title=""):
    # Nube de puntos 3D y la trayectoria ajustada que describen
    data=[
        go.Scatter3d(
            x=point_3D_over_time[:,0], 
            y=point_3D_over_time[:,1], 
            z=point_3D_over_time[:,2], 
            mode="markers", 
            name="",
            marker=dict(size=6, symbol="circle", color="red"),
            text=[str(i + 1) for i in range(point_3D_over_time.shape[0])]
        ),
        go.Scatter3d(
            x=adjusted_point_3D_over_time[:,0], 
            y=adjusted_point_3D_over_time[:,1], 
            z=adjusted_point_3D_over_time[:,2], 
            mode='lines', 
            name='Trajectory',
            line=dict(width=5, color='green')
        )
    ]
    fig = go.Figure(data=data)
    fig.update_layout(
        scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title='z',
            # xaxis = dict(nticks=4, range=[-1.5,1.5]),
            # yaxis = dict(nticks=4, range=[-1.5,1.5]),
            # zaxis = dict(nticks=4, range=[-1.5,1.5])
        ),
        title=dict(
            text=title,
            font=dict(
                family="Courier",
                size=24,
                color="black"
            )
        ), 
        autosize=False,
        width=800, 
        height=500,
        margin=dict(l=40, r=40, b=40, t=40),
        showlegend=False,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            font=dict(
                family="Courier New, monospace",
                size=14,
                color="black"
            ),
            itemsizing="constant"
        ),
        scene_camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=rotation, y=1.25, z=1.25)
        )
    )
    return fig

In [92]:
# Para interactuar con la grafica de posturas
def interact_posture_figure(index, rotation, posture_3D_over_time_list, algorithm_point_names_list, algorithm_connection_list, adjusted_point_3D_over_time_list=None):
    posture_3D_over_time=posture_3D_over_time_list[index]
    fig=get_posture_figure(posture_3D_over_time, algorithm_point_names_list, algorithm_connection_list, adjusted_point_3D_over_time_list, rotation, title="")
    fig.show()

In [93]:
# Para interactuar con la grafica de trayectorias
def interact_trajectory_figure(index, rotation, point_3D_over_time_list, adjusted_point_3D_over_time_list, algorithm_point_names):
    point_3D_over_time=point_3D_over_time_list[index]
    adjusted_point_3D_over_time=adjusted_point_3D_over_time_list[index]
    algorithm_point_name=algorithm_point_names[index]
    fig=get_trajectory_figure(point_3D_over_time, adjusted_point_3D_over_time, rotation, title=algorithm_point_name)
    fig.show()

# Informacion del algoritmo utilizado

In [94]:
algorithm_number_points=33
algorithm_connection_list=[(0,1),(1,2),(2,3),(3,7),(0,4),(4,5),(5,6),(6,8),(9,10),(11,13),(13,15),(15,21),(15,19),(19,17),(17,15),(12,14),(14,16),(16,22),(16,20),(20,18),(18,16),(11,23),(23,24),(24,12),(12,11),(23,25),(25,27),(27,31),(31,29),(29,27),(24,26),(26,28),(28,32),(32,30),(30,28)]
algorithm_point_names_list=[
    "nose",
    "left_eye_inner", "left_eye", "left_eye_outer",
    "right_eye_inner", "right_eye", "right_eye_outer",
    "left_ear", "right_ear",
    "mouth_left", "mouth_right",
    "left_shoulder", "right_shoulder",
    "left_elbow", "right_elbow",
    "left_wrist", "right_wrist",
    "left_pinky_1", "right_pinky_1",
    "left_index_1", "right_index_1",
    "left_thumb_2", "right_thumb_2",
    "left_hip", "right_hip",
    "left_knee", "right_knee",
    "left_ankle", "right_ankle",
    "left_heel", "right_heel",
    "left_foot_index", "right_foot_index"
]

# Directorio de trayectorias
- data
    - trajectory_datasets
        - neutral_posture_XXXX__movement_posture_XXXX
            - data_XXXX.npy
            - ...
        - ...

In [95]:
current_path=os.path.dirname(os.path.abspath('__file__')) 
print(current_path)

/home/chuy/Practicas/PORTAFOLIO/CERTIFICADOS/Samsung Innovation Campus 2024/Proyecto/NEW PROJECT/Notebooks


In [96]:
trajectory_datasets_path=os.path.join("/".join(current_path.split("/")[0:-1]), *["Project", "app", "data", "trajectory_datasets"])
print(trajectory_datasets_path)

/home/chuy/Practicas/PORTAFOLIO/CERTIFICADOS/Samsung Innovation Campus 2024/Proyecto/NEW PROJECT/Project/app/data/trajectory_datasets


In [97]:
print("Lista de directorios de trayectorias:")
folders=os.listdir(trajectory_datasets_path)
folders.sort()
for i,folder_name in enumerate(folders):
    print(f"\t- {folder_name} (indice: {i})")

Lista de directorios de trayectorias:
	- neutral_posture_0001__movement_posture_0001 (indice: 0)
	- neutral_posture_0001__movement_posture_0002 (indice: 1)
	- neutral_posture_0001__movement_posture_0003 (indice: 2)


# Seleccion de trayectoria

In [105]:
trajectory_index=0
specific_trajectory_folder_name=folders[trajectory_index]
specific_trajectory_full_path=os.path.join(trajectory_datasets_path, *[specific_trajectory_folder_name])
print(specific_trajectory_full_path)

/home/chuy/Practicas/PORTAFOLIO/CERTIFICADOS/Samsung Innovation Campus 2024/Proyecto/NEW PROJECT/Project/app/data/trajectory_datasets/neutral_posture_0001__movement_posture_0001


# Obtencion de posturas 3D a lo largo del tiempo de la trayectoria seleccionada

In [106]:
# Posturas 3D a lo largo del tiempo
posture_3D_over_time_list=AdjustTrajectory.get_posture_3D_over_time_list(specific_trajectory_full_path)
# Parametros de los modelos no lineales para cada punto 3D
nonlinear_models_parameters=AdjustTrajectory.load_nonlinear_models_parameters(specific_trajectory_full_path)
print(f"Numero de tomas a lo largo de la trayectoria: {len(posture_3D_over_time_list)}")
print(f"No hay parametros" if nonlinear_models_parameters is None else f"Si hay parametros")

Numero de tomas a lo largo de la trayectoria: 41
Si hay parametros


# Grafica posturas 3D

In [107]:
index_slider=widgets.IntSlider(min=0, max=len(posture_3D_over_time_list) - 1, step=1, value=0)
rotation_slider=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1)
widgets.interact(lambda index,rotation: interact_posture_figure(
    index, 
    rotation, 
    posture_3D_over_time_list, 
    algorithm_point_names_list, 
    algorithm_connection_list,
    None
), index=index_slider, rotation=rotation_slider)

interactive(children=(IntSlider(value=0, description='index', max=40), FloatSlider(value=-1.0, description='ro…

<function __main__.<lambda>(index, rotation)>

# Limpieza de posturas 3D 
Para esta tarea se utilizo un metodo visual para eliminar aquellas posturas que no seguian una trayectoria similar entre la postura anterior y la postura siguiente.

In [78]:
indexes_to_remove=[]
temp=[]
for i in range(len(posture_3D_over_time_list)):
    if i not in indexes_to_remove:
        temp.append(posture_3D_over_time_list[i])
posture_3D_over_time_list=temp
print(f"Numero de tomas a lo largo de la trayectoria: {len(posture_3D_over_time_list)}")

Numero de tomas a lo largo de la trayectoria: 60


# Grafica posturas 3D limpias

In [101]:
index_slider=widgets.IntSlider(min=0, max=len(posture_3D_over_time_list) - 1, step=1, value=0)
rotation_slider=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1)
widgets.interact(lambda index,rotation: interact_posture_figure(
    index, 
    rotation, 
    posture_3D_over_time_list, 
    algorithm_point_names_list, 
    algorithm_connection_list,
    None
), index=index_slider, rotation=rotation_slider)

interactive(children=(IntSlider(value=0, description='index', max=40), FloatSlider(value=-1.0, description='ro…

<function __main__.<lambda>(index, rotation)>

# Calculo de los parametros de los modelos no lineales para cada punto 3D que describen una trayectoria ajustada

In [80]:
# # Calcular parametros de los modelos no lineales de cada punto 3D
# nonlinear_models_parameters=AdjustTrajectory.calculate_nonlinear_models_parameters(posture_3D_over_time_list, algorithm_number_points)
# # Guardar parametros
# AdjustTrajectory.save_nonlinear_models_parameters(specific_trajectory_full_path, nonlinear_models_parameters)

In [108]:
# Cargar parametros
nonlinear_models_parameters=AdjustTrajectory.load_nonlinear_models_parameters(specific_trajectory_full_path)

# Posturas 3D ajustadas a lo largo del tiempo y trayectorias ajustadas de cada punto 3D a lo largo del tiempo

In [109]:
adjusted_posture_3D_over_time_list,adjusted_point_3D_over_time_list=AdjustTrajectory.get_adjusted_posture_3D_and_point_3D_over_time_list(posture_3D_over_time_list, nonlinear_models_parameters, algorithm_number_points, n=100)

# Grafica posturas 3D ajustadas

In [110]:
index_slider=widgets.IntSlider(min=0, max=len(adjusted_posture_3D_over_time_list) - 1, step=1, value=0)
rotation_slider=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1)
widgets.interact(lambda index,rotation: interact_posture_figure(
    index, 
    rotation, 
    adjusted_posture_3D_over_time_list, 
    algorithm_point_names_list, 
    algorithm_connection_list,
    None
), index=index_slider, rotation=rotation_slider)

interactive(children=(IntSlider(value=0, description='index', max=99), FloatSlider(value=-1.0, description='ro…

<function __main__.<lambda>(index, rotation)>

# Grafica posturas 3D ajustadas y trayectorias ajustadas de cada punto 3D

In [111]:
index_slider=widgets.IntSlider(min=0, max=len(adjusted_posture_3D_over_time_list) - 1, step=1, value=0)
rotation_slider=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1)
widgets.interact(lambda index,rotation: interact_posture_figure(
    index, 
    rotation, 
    adjusted_posture_3D_over_time_list, 
    algorithm_point_names_list, 
    algorithm_connection_list,
    adjusted_point_3D_over_time_list
), index=index_slider, rotation=rotation_slider)

interactive(children=(IntSlider(value=0, description='index', max=99), FloatSlider(value=-1.0, description='ro…

<function __main__.<lambda>(index, rotation)>

# Grafica trayectorias ajustadas de cada punto 3D

In [85]:
point_3D_over_time_list=AdjustTrajectory.get_point_3D_over_time_list(posture_3D_over_time_list, algorithm_number_points)
index_slider=widgets.IntSlider(min=0, max=algorithm_number_points - 1, step=1, value=0)
rotation_slider=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1)
widgets.interact(lambda index,rotation: interact_trajectory_figure(
    index, 
    rotation, 
    point_3D_over_time_list, 
    adjusted_point_3D_over_time_list, 
    algorithm_point_names_list
), index=index_slider, rotation=rotation_slider)

interactive(children=(IntSlider(value=0, description='index', max=32), FloatSlider(value=-1.0, description='ro…

<function __main__.<lambda>(index, rotation)>